# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

1.1. Откроем файл с данными.

In [1]:
#импорт бибилиотеки pandas
import pandas as pd
pd.set_option('display.float_format', '{:,.2f}'.format)
#чтение файла с данными и сохранение в переменной credit
credit = pd.read_csv('/datasets/data.csv')

In [2]:
#вывод на экран первые 20 строк таблицы
credit.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"-4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"-5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"-4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"-2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"-6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"-2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


In [3]:
#получение общей информации о данных в таблице credit
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#получение некоторых числовых показателей для столбцов с числовыми значениями
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,"21,525.00","19,351.00","21,525.00","21,525.00","21,525.00","21,525.00","19,351.00"
mean,0.54,"63,046.50",43.29,0.82,0.97,0.08,"167,422.30"
std,1.38,"140,827.31",12.57,0.55,1.42,0.27,"102,971.57"
min,-1.00,"-18,388.95",0.00,0.00,0.00,0.00,"20,667.26"
25%,0.00,"-2,747.42",33.00,1.00,0.00,0.00,"103,053.15"
50%,0.00,"-1,203.37",42.00,1.00,0.00,0.00,"145,017.94"
75%,1.00,-291.10,53.00,1.00,1.00,0.00,"203,435.07"
max,20.00,"401,755.40",75.00,4.00,4.00,1.00,"2,265,604.03"


**Вывод**

На первый взгляд изучая таблицу credit возникли следующие вопросы:

1) Вызвали сомнения данные столбца 'days_employed' (общий трудовой стаж в днях):
- встречаются отрицательные значения
- очень большие числа.

2) Столбец 'education' и 'family_status' заполнен разными регистрами.

3) Столбец 'purpose' - много одинаковых целей кредита записано разными словами.

4) Остальные столбцы сомнений, на первый взгляд, не вызвали.


Изучив общую информацию, делаем выводы:

1) В таблице 12 столбцов.
Согласно документации к данным: children — количество детей в семье, days_employed — общий трудовой стаж в днях, dob_years — возраст клиента в годах, education — уровень образования клиента, education_id — идентификатор уровня образования, family_status — семейное положение, family_status_id — идентификатор семейного положения, gender — пол клиента, income_type — тип занятости, debt — имел ли задолженность по возврату кредитов, total_income — ежемесячный доход, purpose — цель получения кредита.
2) Количество значений в столбцах различается - значит, в данных есть пропущенные значения.
3) Столбец 'days_employed' имеет тип float64 - вещественные числа, что является ошибкой, так как трудовой стаж в днях - тип данных должен быть целые числа. 

Согласно информации некоторых числовых показателей таблицы:
1) В столбце 'children' - количество детей, присутствуют аномальные значения: -1 и 20, которые требуют проверки и замены.
 



## Шаг 2. Предобработка данных

### Обработка пропусков

Проверим наличие пропусков и в каких столбцах. Если доля пропусков от 5-15%% - их необходимо проработать.  


In [5]:
#подсчет пропусков
credit.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
#вывод долевого соотношения количества пропусков
credit.isna().mean()

children           0.00
days_employed      0.10
dob_years          0.00
education          0.00
education_id       0.00
family_status      0.00
family_status_id   0.00
gender             0.00
income_type        0.00
debt               0.00
total_income       0.10
purpose            0.00
dtype: float64

Пропуски обнаружены в столбцах: 'days_employed' и 'total_income'. Причем их количество по столбцам одинаковое. 

Долевое соотношение равно 10%, поэтому их необходимо проработать.

Разумнее всего:
- в столбце 'days_employed' пропуски заменить на медианное значение этого столбца в зависимости от возраста, чтобы избежать возможных выбросов в столбце, чего не сможет избежать просто среднее значение по столбцу и дать более искаженные от действительности данные;
- в столбце 'total_income' так же пропуски заменить на медианное значение этого столбца в зависимости от типа занятости.


Заменим отрицательные значения столбца 'days_employed' на положительные, чтобы использовать  их при дальнейших рассчетах.

In [7]:
#поменяем отрицательные числа в столбце 'days_employed' на положиткльные 
credit['days_employed'] = credit['days_employed'].apply(lambda x: x if x>0 else
x*-1)
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


In [8]:
#выведем на экран первые 10 строк с пропусками в столбцах 'days_employed'

credit[credit['days_employed'].isnull()].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,nan,65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу
26,0,nan,41,среднее,1,женат / замужем,0,M,госслужащий,0,nan,образование
29,0,nan,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,nan,строительство жилой недвижимости
41,0,nan,50,среднее,1,женат / замужем,0,F,госслужащий,0,nan,сделка с подержанным автомобилем
55,0,nan,54,среднее,1,гражданский брак,1,F,пенсионер,1,nan,сыграть свадьбу
65,0,nan,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,nan,операции с коммерческой недвижимостью
67,0,nan,52,высшее,0,женат / замужем,0,F,пенсионер,0,nan,покупка жилья для семьи
72,1,nan,32,высшее,0,женат / замужем,0,M,госслужащий,0,nan,операции с коммерческой недвижимостью
82,2,nan,50,высшее,0,женат / замужем,0,F,сотрудник,0,nan,жилье
83,0,nan,52,среднее,1,женат / замужем,0,M,сотрудник,0,nan,жилье


Заменим пропуски в столбце 'days_employed' на медианное значение этого столбца в зависимости от категории возраста (столбец 'dob_years')

In [9]:


def credit_nan(credit, value, category):
    '''
    Создадим функцию:
    принимает 3 аргумента:
    credit - датасет
    value - столбец значений
    category - столбец категорий.

    Циклом перебирает все уникальные значения столбца category 
    и пропишем условия -
    отобрази нам те строки, в которых есть пропуск и 
    где тип category равен уникальному значению столбца category 
    и замени значение value на медиану значений этой категории 
    '''
    for type_unique in credit[category].unique():
        credit.loc[(credit[value].isna())&(credit[category] ==type_unique), value] = \
        credit.loc[credit[category] ==type_unique, value].median()
    return credit
credit = credit_nan(credit, 'days_employed', 'dob_years')
credit.head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


In [10]:
#проверим замену пропусков в столбце 'days_employed'

credit[credit['days_employed'].isnull()].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Проверим адекватность значений столбца 'days_employe':
переведем значения столбца, которые указаны в днях в годы

In [11]:
#переведем значения столбца 'days_employed' в годы и сохраним в отдельный столбец 'days_employed_years'
credit['days_employed_years'] = credit['days_employed'].apply(lambda x: x/365)


In [12]:
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
0,1,"8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья,23.12
1,1,"4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля,11.03
2,0,"5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья,15.41
3,3,"4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование,11.30
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу,932.24
5,0,926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья,2.54
6,0,"2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем,7.89
7,0,152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование,0.42
8,2,"6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы,18.99
9,0,"2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи,6.00


В анализе столбца 'days_employed' замечены аномальный срок стажа - более 900 лет. Выведем такие аномальные сроки, например, где стаж более 27375 дней (75 лет). 

In [13]:
#выведем строки, где трудовой стаж более 27375 дней
credit.loc[credit['days_employed']>27375]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу,932.24
12,0,"357,103.97",65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу,978.37
18,0,"400,281.14",53,среднее,1,вдовец / вдова,2,F,пенсионер,0,"56,823.78",на покупку подержанного автомобиля,"1,096.66"
24,1,"338,551.95",57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,"290,547.24",операции с коммерческой недвижимостью,927.54
25,0,"363,548.49",67,среднее,1,женат / замужем,0,M,пенсионер,0,"55,112.76",покупка недвижимости,996.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,"338,904.87",53,среднее,1,гражданский брак,1,M,пенсионер,0,"75,439.99",сыграть свадьбу,928.51
21508,0,"386,497.71",62,среднее,1,женат / замужем,0,M,пенсионер,0,"72,638.59",недвижимость,"1,058.90"
21509,0,"362,161.05",59,высшее,0,женат / замужем,0,M,пенсионер,0,"73,029.06",операции с недвижимостью,992.22
21518,0,"373,995.71",59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,"153,864.65",сделка с автомобилем,"1,024.65"


На первый взгляд аномально большие значения трудового стажа встречаются у категории "пенсиорнер". Проверим эту гипотезу.

In [14]:
credit.loc[credit['days_employed']>27375].groupby('income_type', as_index=False)[['days_employed_years']].count()

,income_type,days_employed_years
0,безработный,2
1,госслужащий,13
2,компаньон,26
3,пенсионер,3733
4,предприниматель,1
5,сотрудник,78


Действительно, самое большое количество аномальных значений трудового стажа встречается в категории пенсионер, но так же встречаются и в других категориях типа занятости.
Заменим все подобные аномальные значения на медианное значение по всему столбцу 'days_employed'. В связи с такими неадекватно большими аномальными значениями, замена их на медиану будет разумнее всего.

In [15]:
#найдем медиану столбца 'days_employed'
median_days_employed = credit['days_employed'].median()
display(median_days_employed)

2170.569747334712

In [16]:
#заменим все значения трудового стажа, который больше 27375 дней (75 лет), на медиану столбца 'days_employed'
credit.loc[(credit['days_employed']>27375),'days_employed'] = \
credit.loc[(credit['days_employed']>27375),'days_employed'] = median_days_employed

In [17]:
#проверим замену строк 
credit.loc[credit['days_employed']>27375]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years


In [18]:
#проверим датасет
credit.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
7142,0,"7,720.71",50,среднее,1,Не женат / не замужем,4,F,госслужащий,0,"168,337.71",операции с недвижимостью,21.15
15607,0,"1,913.62",46,среднее,1,гражданский брак,1,F,сотрудник,0,"159,343.54",свадьба,5.24
16938,0,"3,108.18",27,высшее,0,Не женат / не замужем,4,M,сотрудник,0,"654,098.42",операции с жильем,8.52
4864,2,777.35,27,высшее,0,женат / замужем,0,F,сотрудник,0,"158,522.44",жилье,2.13
6721,4,"1,181.11",36,высшее,0,женат / замужем,0,F,госслужащий,0,"154,701.84",покупка жилья,3.24
17818,1,"2,170.57",64,высшее,0,гражданский брак,1,F,пенсионер,0,"115,727.16",на проведение свадьбы,"1,036.74"
9135,1,"1,078.81",36,среднее,1,вдовец / вдова,2,F,сотрудник,0,"246,226.36",строительство жилой недвижимости,2.96
1520,0,"3,650.01",53,начальное,3,женат / замужем,0,F,сотрудник,0,nan,операции со своей недвижимостью,10.00
9068,0,"2,203.08",47,высшее,0,женат / замужем,0,F,сотрудник,0,nan,покупка жилья для сдачи,6.04
9239,0,"1,520.41",52,среднее,1,женат / замужем,0,M,сотрудник,0,"135,370.90",заняться высшим образованием,4.17


Теперь проработаем пропуски столбца 'total_income', заменив их на медианное значение столбца 'total_income' в зависимости от типа занятости 'income_type'. 

In [19]:
#выведем на экран первые 10 строк с пропусками в столбцах 'total_income'

credit[credit['total_income'].isnull()].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
12,0,"2,170.57",65,среднее,1,гражданский брак,1,M,пенсионер,0,nan,сыграть свадьбу,978.37
26,0,"1,864.66",41,среднее,1,женат / замужем,0,M,госслужащий,0,nan,образование,5.11
29,0,"2,170.57",63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,nan,строительство жилой недвижимости,982.14
41,0,"2,626.99",50,среднее,1,женат / замужем,0,F,госслужащий,0,nan,сделка с подержанным автомобилем,7.20
55,0,"4,026.54",54,среднее,1,гражданский брак,1,F,пенсионер,1,nan,сыграть свадьбу,11.03
65,0,618.73,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,nan,операции с коммерческой недвижимостью,1.70
67,0,"3,395.64",52,высшее,0,женат / замужем,0,F,пенсионер,0,nan,покупка жилья для семьи,9.30
72,1,"1,446.62",32,высшее,0,женат / замужем,0,M,госслужащий,0,nan,операции с коммерческой недвижимостью,3.96
82,2,"2,626.99",50,высшее,0,женат / замужем,0,F,сотрудник,0,nan,жилье,7.20
83,0,"3,395.64",52,среднее,1,женат / замужем,0,M,сотрудник,0,nan,жилье,9.30


In [20]:
#вызовем функцию для замены пропусков на медианное значение в разбивки по категориям типа занятости
credit = credit_nan(credit, 'total_income', 'income_type')
credit.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
0,1,"8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья,23.12
1,1,"4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля,11.03
2,0,"5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья,15.41
3,3,"4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование,11.30
4,0,"2,170.57",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу,932.24
5,0,926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья,2.54
6,0,"2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем,7.89
7,0,152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование,0.42
8,2,"6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы,18.99
9,0,"2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи,6.00


In [21]:
#проверим замену пропусков в столбце 'total_income'

credit[credit['total_income'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years


In [22]:
# еще раз проверим отсутствие пропусков по всему датасету
credit.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
days_employed_years    0
dtype: int64

В столбце 'dob_years' были обнаружены нулевые значения - что противоречит здравому смыслу. Сделаем замену этих нулевых значений на значение медианы по данному столбцу, так как возраст варьируется от студента до пенсионера, то медиана будет ближе к действительности.

In [23]:
#выведем на экран строки, где значение столбца 'dob_years' равны 0 
display(credit[credit['dob_years'] == 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
99,0,"2,170.57",0,Среднее,1,женат / замужем,0,F,пенсионер,0,"71,291.52",автомобиль,949.43
149,0,"2,664.27",0,среднее,1,в разводе,3,F,сотрудник,0,"70,176.44",операции с жильем,7.30
270,3,"1,872.66",0,среднее,1,женат / замужем,0,F,сотрудник,0,"102,166.46",ремонт жилью,5.13
578,0,"2,170.57",0,среднее,1,женат / замужем,0,F,пенсионер,0,"97,620.69",строительство собственной недвижимости,"1,090.02"
1040,0,"1,158.03",0,высшее,0,в разводе,3,F,компаньон,0,"303,994.13",свой автомобиль,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,"1,759.04",0,среднее,1,женат / замужем,0,F,сотрудник,0,"142,594.40",жилье,4.82
20462,0,"2,170.57",0,среднее,1,женат / замужем,0,F,пенсионер,0,"259,193.92",покупка своего жилья,928.04
20577,0,"2,170.57",0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,"129,788.76",недвижимость,908.88
21179,2,108.97,0,высшее,0,женат / замужем,0,M,компаньон,0,"240,702.01",строительство жилой недвижимости,0.30


In [24]:
median_dob_years = credit['dob_years'].median()
display(median_dob_years)

42.0

In [25]:
#заменим все нулевые значения в столбце dob_years на медиану по этому столбцу
credit.loc[(credit['dob_years'] == 0),'dob_years'] = credit.loc[(credit['dob_years'] == 0),'dob_years'] = median_dob_years
   

In [26]:
credit['dob_years'].value_counts()

42.00    698
35.00    617
40.00    609
41.00    607
34.00    603
38.00    598
33.00    581
39.00    573
31.00    560
36.00    555
44.00    547
29.00    545
30.00    540
48.00    538
37.00    537
50.00    514
43.00    513
32.00    510
49.00    508
28.00    503
45.00    497
27.00    493
56.00    487
52.00    484
47.00    480
54.00    479
46.00    475
58.00    461
57.00    460
53.00    459
51.00    448
59.00    444
55.00    443
26.00    408
60.00    377
25.00    357
61.00    355
62.00    352
63.00    269
64.00    265
24.00    264
23.00    254
65.00    194
66.00    183
22.00    183
67.00    167
21.00    111
68.00     99
69.00     85
70.00     65
71.00     58
20.00     51
72.00     33
19.00     14
73.00      8
74.00      6
75.00      1
Name: dob_years, dtype: int64

In [27]:
#проверим:
display(credit[credit['dob_years'] == 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years


В столбце 'children' обнаружены аномальные значения: -1 и 20. Проработаем их. 
1) Отрицательные значения, равные: -1, заменим на положительные: 1, чтобы минус не мешал в дальнейших вычислениях.
2) Найдем среднее значение по столбцу.
3) Так как в столбце нет больших колебаний значений - заменим аномальные значения, равные 20, на среднее значение по столбцу.

In [28]:
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,days_employed_years
count,"21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00"
mean,0.54,"2,309.40",43.49,0.82,0.97,0.08,"165,225.32",183.38
std,1.38,"2,021.19",12.22,0.55,1.42,0.27,"98,043.67",379.82
min,-1.00,24.14,19.00,0.00,0.00,0.00,"20,667.26",0.07
25%,0.00,"1,007.37",34.00,1.00,0.00,0.00,"107,798.17",2.76
50%,0.00,"2,170.57",42.00,1.00,0.00,0.00,"142,594.40",5.95
75%,1.00,"2,636.77",53.00,1.00,1.00,0.00,"195,549.94",15.04
max,20.00,"18,388.95",75.00,4.00,4.00,1.00,"2,265,604.03","1,100.70"


In [29]:
# заменим отрицательные значения: -1 в столбце 'children' на положительные: 1
credit['children'] = credit['children'].apply(lambda x: x if x>0 else
x*-1)
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,days_employed_years
count,"21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00"
mean,0.54,"2,309.40",43.49,0.82,0.97,0.08,"165,225.32",183.38
std,1.38,"2,021.19",12.22,0.55,1.42,0.27,"98,043.67",379.82
min,0.00,24.14,19.00,0.00,0.00,0.00,"20,667.26",0.07
25%,0.00,"1,007.37",34.00,1.00,0.00,0.00,"107,798.17",2.76
50%,0.00,"2,170.57",42.00,1.00,0.00,0.00,"142,594.40",5.95
75%,1.00,"2,636.77",53.00,1.00,1.00,0.00,"195,549.94",15.04
max,20.00,"18,388.95",75.00,4.00,4.00,1.00,"2,265,604.03","1,100.70"


In [30]:
#найдем среднее значение по столбцу 'children' и сохраним ее в переменной mean_children
mean_children = round(credit['children'].mean())
display(mean_children)

1

In [31]:
#выведем на экран строки, где значение столбца 'children' равны: 20 
display(credit[credit['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
606,20,880.22,21.00,среднее,1,женат / замужем,0,M,компаньон,0,"145,334.87",покупка жилья,2.41
720,20,855.60,44.00,среднее,1,женат / замужем,0,F,компаньон,0,"112,998.74",покупка недвижимости,2.34
1074,20,"3,310.41",56.00,среднее,1,женат / замужем,0,F,сотрудник,1,"229,518.54",получение образования,9.07
2510,20,"2,714.16",59.00,высшее,0,вдовец / вдова,2,F,сотрудник,0,"264,474.84",операции с коммерческой недвижимостью,7.44
2941,20,"2,161.59",42.00,среднее,1,женат / замужем,0,F,сотрудник,0,"199,739.94",на покупку автомобиля,5.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,"1,240.26",40.00,среднее,1,женат / замужем,0,F,сотрудник,1,"133,524.01",свой автомобиль,3.40
21325,20,601.17,37.00,среднее,1,женат / замужем,0,F,компаньон,0,"102,986.07",профильное образование,1.65
21390,20,"3,650.01",53.00,среднее,1,женат / замужем,0,M,компаньон,0,"172,357.95",покупка жилой недвижимости,10.00
21404,20,494.79,52.00,среднее,1,женат / замужем,0,M,компаньон,0,"156,629.68",операции со своей недвижимостью,1.36


In [32]:
credit.loc[(credit['children'] == 20),'children'] = credit.loc[(credit['children'] == 20),'children'] = mean_children

In [33]:
#проверим еще раз числовые показатели по таблицы
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,days_employed_years
count,"21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00","21,525.00"
mean,0.48,"2,309.40",43.49,0.82,0.97,0.08,"165,225.32",183.38
std,0.75,"2,021.19",12.22,0.55,1.42,0.27,"98,043.67",379.82
min,0.00,24.14,19.00,0.00,0.00,0.00,"20,667.26",0.07
25%,0.00,"1,007.37",34.00,1.00,0.00,0.00,"107,798.17",2.76
50%,0.00,"2,170.57",42.00,1.00,0.00,0.00,"142,594.40",5.95
75%,1.00,"2,636.77",53.00,1.00,1.00,0.00,"195,549.94",15.04
max,5.00,"18,388.95",75.00,4.00,4.00,1.00,"2,265,604.03","1,100.70"


**Вывод**

Итак, в процессе предобработки данных были обнаружены пропуски в столбцах: 'days_employed' и 'total_income'. Причем их количество по столбцам было одинаковое. 

Долевое соотношение равно 10%, поэтому их необходимо было проработать.

- В столбце 'days_employed' пропуски заменили на медианное значение этого столбца в зависимости от возраста (столбец 'dob_years'), чтобы избежать возможных выбросов в столбце, чего не сможет избежать просто среднее значение по столбцу и может дать более искаженные от действительности данные, в отличии от медианы.
-В столбце 'total_income' так же пропуски заменить на медианное значение этого столбца в зависимости от типа занятости 'income_type'.

В столбце 'days_employed' заменили отрицательные значения на положительные, чтобы использовать  их при дальнейших расчетах.

В анализе столбца 'days_employed' замечены аномальный срок стажа - более 900 лет.  Заменили все подобные аномальные значения на медианное значение по всему столбцу 'days_employed'. В связи с такими неадекватно большими колебаниями значений, замена их на медиану была разумнее всего.

Так же в столбце  возраст (’dob_years’)  были обнаружены нулевые значения - что противоречит здравому смыслу. Заменили эти нулевые значения на значение медианы по данному столбцу, так как возраст варьируется от студента до пенсионера, поэтому медиана будет ближе к действительности.

В столбце 'children' обнаружены так же неадекватные значения: -1 и 20.  
-Отрицательные значения, равные: -1, заменили на положительные: 1, чтобы минус не мешал в дальнейших вычислениях.
-Так как в столбце нет больших колебаний значений - заменили значения, равные 20, на среднее значение по столбцу.



### Замена типа данных

In [34]:
#получение общей информации о данных в таблице credit
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children               21525 non-null int64
days_employed          21525 non-null float64
dob_years              21525 non-null float64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null float64
purpose                21525 non-null object
days_employed_years    21525 non-null float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


Анализируя описание данных по столбцам и их типы, можно сделать вывод о замене типа данных:
- столбец "общий трудовой стаж в днях" ('days_employed') тип float64 заменить на int, так как дни - целое число;
- столбец "возраст клиента в годах" ('dob_years') тип float64 заменить на int, так как годы - целое число;
- столбец "ежемесячный доход" ('total_income') тип float64 заменить на int, так как для анализа проще будет работать с целыми числами, а точность до копеек никак не может повлиять на точность результата;



In [35]:
#переведем тип значений столбца days_employed в тип int (целые числа)
'''
Создадим функцию с одним аргументом:
value - значение,
которая возвращает целое число
'''
def temp_1 (value):
    return int(value)
credit['days_employed'] = credit['days_employed'].apply(temp_1)

#проверим результат
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children               21525 non-null int64
days_employed          21525 non-null int64
dob_years              21525 non-null float64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null float64
purpose                21525 non-null object
days_employed_years    21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


In [36]:
#переведем тип значений столбца dob_years в тип int (целые числа)
credit['dob_years'] = credit['dob_years'].apply(temp_1)

#переведем тип значений столбца total_income в тип int (целые числа)
credit['total_income'] = credit['total_income'].apply(temp_1)

#проверим результат
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children               21525 non-null int64
days_employed          21525 non-null int64
dob_years              21525 non-null int64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null int64
purpose                21525 non-null object
days_employed_years    21525 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Итак, в столбцах :

- "общий трудовой стаж в днях" ('days_employed')
- "возраст клиента в годах" ('dob_years') 
- "ежемесячный доход" ('total_income') 

был заменен тип данных на int (целые числа), что соответствует здравому смыслу и упрощения дальнейших рассчетов.

### Обработка дубликатов

Для обработки дубликатов сначала приведем все значения к нижнему регистру.

In [37]:
#подсчет явных дубликатов
credit.duplicated().sum()

54

In [38]:
#соберем все уникальные совпадения столбца 'education'
credit['education'].value_counts()


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [39]:
# все данные столбца 'education'приведем к нижнему регистру
    
credit['education']=credit['education'].str.lower()    


In [40]:
#проверим, что все значения в столбце 'education' переведены в нижний регистр 
credit['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [41]:
#соберем все уникальные совпадения столбца 'family_status'
credit['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [42]:
#все данные столбца 'family_status'приведем к нижнему регистру
credit['family_status']=credit['family_status'].str.lower()    



In [43]:
#проверим, что все значения в столбце 'family_status' переведены в нижний регистр 
credit['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [44]:
#проверим наличие уникальных совпадений столбец 'gender'
credit['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [45]:
#выведем на экран строку со значением в столбце 'gender' - XNA
credit[credit['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости,6.46


In [46]:
#замена ячейки в столбце 'gender': XNA на M
credit['gender'] = credit['gender'].str.replace('XNA', 'M')

In [47]:
#проверим исправление в столбеце 'gender'
credit['gender'].value_counts()


F    14236
M     7289
Name: gender, dtype: int64

In [48]:
#проверим наличие уникальных совпадений столбец 'income_type'
credit['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [49]:
credit[credit['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years


In [50]:
#соберем все уникальные совпадения столбца 'purpose'
credit['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [51]:
#подсчет явных дубликатов
credit.duplicated().sum()

71

In [52]:
#удаление дубликатов (с удалением старых индексов и форматированием новых)
credit = credit.drop_duplicates().reset_index(drop=True)

In [53]:
#проверка на отсутсие дубликатов
credit.duplicated().sum()

0

**Вывод**
Сначала привели все данные в таблице к нижнему регистру, чтобы точнее отфильтровать дубликаты. Удалили дубликаты. 

### Лемматизация

Проведем лемматизацию столбца 'purpose':

1) проведем лемматизацию столбца 'purpose';

2) сохраним результат лемматизации в отдельный столбец.

In [54]:
# импортируем pymystem3 
from pymystem3 import Mystem
m = Mystem() 

#соберем все уникальные совпадения столбца 'purpose' и после: 
#вытащим все индексы получившейся таблицы в отдельный столбец
credit_grouped = credit['purpose'].value_counts().reset_index()

#создадим функцию лемматизации
def credit_lem(i): 
    lemmas = m.lemmatize(i)
    return lemmas

#создадим новый столбец лемматизированных значений столбца 'purpose'
credit['credit_lem'] = credit['purpose'].apply(credit_lem)

#вывод первых 20-ти строк получившейся таблицы
credit.head(20)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]"
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2.54,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7.89,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0.42,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18.99,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,6.00,"[покупка, , жилье, , для, , семья, \n]"


**Вывод**
Создали отдельный столбец 'credit_lem' c лемматизированными значениями столбца 'purpose'

### Категоризация данных


1) Рассмотрим зависимость данных в столбцах: education и education_id, а так же family_status и family_status_id.

2) Разобьем по категориям данные столбца purpose (цель кредита) и выявить самую популярную покупку в кредит.

In [55]:
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]"
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]"


In [56]:
#вывод уникальных значений столбца 'education'
credit['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [57]:
#вывод таблицы из двух столбцов 'education','education_id', сохранить в переменной credit_educ
credit_educ = credit[['education','education_id']]
display(credit_educ)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21449,среднее,1
21450,среднее,1
21451,среднее,1
21452,среднее,1


In [58]:
#группируем по столбцу 'education' и выводим среднее столбца 'education_id'
credit_educ.groupby('education')['education_id'].mean()

education
высшее                 0
начальное              3
неоконченное высшее    2
среднее                1
ученая степень         4
Name: education_id, dtype: int64

Итак, можно сделать вывод о зависимости столбцов 'education' (уровень образования клиента) и 'education_id' (идентификатор уровня образования клиента):
- высшее соответствует цифра 0
- среднее - соотвтетствует цифра 1
- неоконченное высшее - соответствует цифра 2
- начальное - соотвтетствует цифра 3
- ученая степень - соответствует цифра 4

По аналогии проведем анализ зависимости между столбцами family_status и family_status_id.

In [59]:
#вывод таблицы из столбцов 'family_status','family_status_id' и сохранить в переменной family_id
family_id = credit[['family_status','family_status_id']]


display(family_id)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21449,гражданский брак,1
21450,женат / замужем,0
21451,гражданский брак,1
21452,женат / замужем,0


In [60]:
#группируем по столбцу 'family_status' и выводим среднее столбца 'family_status_id'
family_id.groupby('family_status')['family_status_id'].mean()

family_status
в разводе                3
вдовец / вдова           2
гражданский брак         1
женат / замужем          0
не женат / не замужем    4
Name: family_status_id, dtype: int64

Итак, делаем вывод о зависимости столбцов 'family_status' (семейное положение) и 'family_status_id' (идентификатор семейного положения):
- в разводе - соответствует 3
- вдовец / вдова - соответствет 2
- гражданский брак - 1
- женат / замужем - 0
- не женат / не замужем - 4

Проанализируем столбец 'purpose' (цели кредита) и выявим самые популярные у заемщиков покупки в кредит.

In [61]:
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]"
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]"


In [62]:
def purpose_res(row):
    """"   
    создание функции, которая возвращает "цель кредита" в одно слово, по правилам:
    -если в строке есть лемма 'свадьба' - вернуть слово 'свадьба'
    -если в строке есть лемма 'недвижимость' - вернуть слово 'недвижимость'
    -если в строке есть лемма 'жилье' - вернуть слово 'недвижимость'
    -если в строке есть лемма 'автомобиль' - вернуть слово 'автомобиль'
    -если в строке есть лемма 'образование' - вернуть слово 'образование'
    -в остальных вариантах - вернуть слово 'прочие цели'
    """
    
    if 'свадьба' in row:
        return 'свадьба'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'образование' in row:
        return 'образование'
    
    return 'прочие цели'    
        


#создание нового столбца с резульатами функции debt_purpose_res: 
credit['purpose_res'] = credit['credit_lem'].apply(purpose_res)

# вывод полученных отформатированных категорий "целей кредита"
credit.head()      

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem,purpose_res
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]",образование
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]",свадьба


In [63]:
#группируем таблицу по столбцу 'purpose_res'и подсчитываем количество каждой группы 
#и сохраним в переменной purpose_res_gr
purpose_res_gr = credit.groupby(['purpose_res']).agg({'purpose_res':'count'})
display(purpose_res_gr)



,purpose_res
purpose_res,
автомобиль,4306
недвижимость,10811
образование,4013
свадьба,2324


In [64]:
#отсортируем таблицу purpose_res_gr
#purpose_res_gr.sort_values(by=['purpose_res'],ascending=False).reset_index(drop=True)
purpose_res_gr['purpose_res'].sort_values(ascending=False)

purpose_res
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_res, dtype: int64

Итак, самая популярная покупка в кредит - это покупка недвижимости. 

**Вывод**

Провели анализ завсисмости столбцов education и education_id, а так же family_status и family_status_id. Выявили следующие соответствия:

1) 'education' (уровень образования клиента) и 'education_id' (идентификатор уровня образования клиента):
- высшее соответствует цифра 0
- среднее - соотвтетствует цифра 1
- неоконченное высшее - соответствует цифра 2
- начальное - соотвтетствует цифра 3
- ученая степень - соответствует цифра 4

2) 'family_status' (семейное положение) и 'family_status_id' (идентификатор семейного положения):
- в разводе - соответствует 3
- вдовец / вдова - соответствет 2
- гражданский брак - 1
- женат / замужем - 0
- не женат / не замужем - 4

Сгруппировали по категориям столбец purpose (цели кредита):
- недвижимость    10811
- автомобиль       4306
- образование      4013
- свадьба        
и выяснили, что самой популярной покупкой в кредит у населения является покупка недвижимости.

### Ответы на вопросы.

2.6.1. НАЛИЧИЕ ДЕТЕЙ И ВОЗВРАТ КРЕДИТА В СРОК

Есть ли зависимость между наличием детей и возвратом кредита в срок? 

1) Сгруппируем всех заемщиков на группы:
- надежный бездетный заёмщик      
- надежный заёмщик с детьми        
- НЕнадежный бездетный заёмщик     
- НЕнадежный заёмщик с детьми 

2) Найдем долю ненадежных заемщиков в каждой группе



In [65]:
def debt_children_res(row):
  
    """
    создание функции, которая возвращает деление заемщиков на категории, по правилам:
     - 'надежный бездетный заёмщик': debt = 0, children = 0  
     - 'надежный заёмщик с детьми': debt = 0, children != 0
     - 'НЕнадежный заёмщик с детьми': debt = 1, children != 0
     - 'НЕнадежный бездетный заёмщик' - в остальных случаях
    """
 
    debt = row['debt']
    children = row['children']
    
    if debt == 0:
        if children == 0:
            return 'надежный бездетный заёмщик'
    if debt == 0:
        if children != 0:
            return 'надежный заёмщик с детьми'
    if debt == 1:
        if children!= 0:
            return 'НЕнадежный заёмщик с детьми'
    
    return 'НЕнадежный бездетный заёмщик'

#проверка работы функции debt_children_res(row):
row_values = [1, 1]
row_columns = ['debt', 'children']
row = pd.Series(data=row_values, index=row_columns)
debt_children_res(row)

#создание нового столбца с резульатами функции debt_children_res(row): 
credit['debt_children_res'] = credit.apply(debt_children_res, axis=1)

# вывод полученных категорий заемщиков и подсчет их количества
display(credit['debt_children_res'].value_counts())

надежный бездетный заёмщик      13028
надежный заёмщик с детьми        6685
НЕнадежный бездетный заёмщик     1063
НЕнадежный заёмщик с детьми       678
Name: debt_children_res, dtype: int64

In [66]:
#сохраним в переменной:
#x_0 - количество всех 'надежный бездетный заёмщик'
#x_0_ch - количество всех 'надежный заёмщик с детьми'
#x_1 - количество всех 'НЕнадежный бездетный заёмщик'
#x_1_ch - количество всех 'НЕнадежный заёмщик с детьми'
x_0 = credit.loc[credit['debt_children_res'] == 'надежный бездетный заёмщик', 'debt_children_res'].count()
display(x_0)
x_0_ch = credit.loc[credit['debt_children_res'] == 'надежный заёмщик с детьми', 'debt_children_res'].count()
display(x_0_ch)
x_1 = credit.loc[credit['debt_children_res'] == 'НЕнадежный бездетный заёмщик', 'debt_children_res'].count()
display(x_1)
x_1_ch = credit.loc[credit['debt_children_res'] == 'НЕнадежный заёмщик с детьми', 'debt_children_res'].count()
display(x_1_ch)

13028

6685

1063

678

In [67]:
#доля ненадежных у бездетных
display(x_1/(x_0+x_1))

0.07543822297920659

In [68]:
#доля ненадежных у заемщика с детьми
display(x_1_ch/(x_1_ch+x_0_ch))

0.09208203178052424

**Вывод 2.6.1.**

Доля ненадежных заемщиков (заещиков, у которых был факт невозврата кредита в срок):
 - у бездетных составила - 0.075 (7,5%)
 - у заемщиков с детьми - 0.092 (9,2%)

Таким образом, среди заемщиков с детьми процент невозврата кредита в срок больше, чем у бездетных заемщиков.  

2.6.2. СЕМЕЙНОЕ ПОЛОЖЕНИЕ И ВОЗВРАТ КРЕДИТА В СРОК

Есть ли зависимость между семейным положением и возвратом кредита в срок?

1) Сгруппируем заемщиков по категориям:
- надежный НЕодинокий заёмщик      
- надежный одинокий заёмщик         
- НЕнадежный НЕодинокий заёмщик     
- НЕнадежный одинокий заёмщик

2) Найдем долю ненадежных заемщиков в каждой группе

In [69]:
#вывод столбцов 'family_status_id','debt' и сохранение данной таблицы в переменной debt_family_status
debt_family_status = credit[['family_status_id','debt']]

#вывод таблицы debt_family_status на экран
display(debt_family_status)

,family_status_id,debt
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
21449,1,0
21450,0,0
21451,1,1
21452,0,1


In [70]:
#вывод на экран уникальных значений столбца 'family_status_id' и 'family_status' и подсчет их количества
display(debt_family_status['family_status_id'].value_counts())
display(credit['family_status'].value_counts())

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [71]:
def debt_family_status_res(row):
    """"
    создание функции, которая возвращает деление заемщиков на категории, по правилам:
    -'надежный одинокий заёмщик': debt = 0, family_status_id >= 2
    -'надежный НЕодинокий заёмщик ': debt = 0, family_status_id < 2
    -'НЕнадежный одинокий заёмщик': debt = 1, family_status_id >= 2
    -'НЕнадежный НЕодинокий заёмщик' во всех остальных случаях
    """
 
    debt = row['debt']
    family_status_id = row['family_status_id']
    
    if debt == 0:
        if family_status_id >= 2:
            return 'надежный одинокий заёмщик'
    if debt == 0:
        if family_status_id < 2:
            return 'надежный НЕодинокий заёмщик '
    if debt == 1:
        if family_status_id >= 2:
            return 'НЕнадежный одинокий заёмщик'
    
    return 'НЕнадежный НЕодинокий заёмщик'

#проверка работы функции debt_family_status_res(row):
row_values = [1, 1]
row_columns = ['debt', 'family_status_id']
row = pd.Series(data=row_values, index=row_columns)
debt_family_status_res(row)

#создание нового столбца с резульатами функции debt__family_status_res(row): 
credit['result'] = credit.apply(debt_family_status_res, axis=1)

# вывод полученных категорий заемщиков и подсчет их количества
display(credit['result'].value_counts())

надежный НЕодинокий заёмщик      15171
надежный одинокий заёмщик         4542
НЕнадежный НЕодинокий заёмщик     1319
НЕнадежный одинокий заёмщик        422
Name: result, dtype: int64

In [72]:
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem,purpose_res,debt_children_res,result
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]",недвижимость,надежный заёмщик с детьми,надежный НЕодинокий заёмщик
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]",автомобиль,надежный заёмщик с детьми,надежный НЕодинокий заёмщик
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]",недвижимость,надежный бездетный заёмщик,надежный НЕодинокий заёмщик
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]",образование,надежный заёмщик с детьми,надежный НЕодинокий заёмщик
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]",свадьба,надежный бездетный заёмщик,надежный НЕодинокий заёмщик


In [73]:
#сохраним в переменной:
#y_0 - количество всех 'надежный одинокий заёмщик'
#y_0_wed - количество всех 'надежный НЕодинокий заёмщик'
#y_1 - количество всех 'НЕнадежный одинокий заёмщик'
#y_1_wed - количество всех 'НЕнадежный НЕодинокий заёмщик'

y_0 = credit.loc[credit['result'] == 'надежный одинокий заёмщик', 'result'].count()
display(y_0)
y_0_wed = credit.loc[credit['result'] == 'надежный НЕодинокий заёмщик ', 'result'].count()
display(y_0_wed)
y_1 = credit.loc[credit['result'] == 'НЕнадежный одинокий заёмщик', 'result'].count()
display(y_1)
y_1_wed = credit.loc[credit['result'] == 'НЕнадежный НЕодинокий заёмщик', 'result'].count()
display(y_1_wed)



4542

15171

422

1319

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> 

Почему ты делаешь эту категоризацию в этом пункте, у тебя же есть соответствующий? Перенеси ее туда - аналогично со другими столбцами.
</div>



Таким образом: 
0 - женат / замужем; 
1 - гражданский брак;
2 - вдовец / вдова;
3 - в разводе; 
4 - не женат / не замужем.

In [74]:
#доля ненадежных среди неодиноких заемщиков
display(y_1_wed/(y_0_wed+y_1_wed))

0.07998787143723468

In [75]:
#доля ненадежных среди одиноких заемщиков
display(y_1/(y_0+y_1))

0.08501208702659147

***Вывод 2.6.2.***

Таким образом:
- доля ненадежных среди неодиноких заемщиков - 0.08 (8%)
- доля ненадежных среди одиноких заемщиков - 0.09 (9%)

То есть ненадежных среди одиноких заемщиков, хотя и не на много, но все таки больше.

2.6.3. УРОВЕНЬ ДОХОДА И ВОЗВРАТ КРЕДИТА В СРОК

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

1) Сгруппируем заемщиков на группы:
- надежный заемщик с доходом средним и ниже среднего      
- надежный заемщик с доходом выше среднего               
- НЕнадежный заемщик с доходом средним и ниже среднего    
- НЕнадежный заемщик с доходом выше среднего   

    1.1) Средним обозначим - медиану по столбцу 'total_income'

2) Найдем долю ненадежных заемщиков в каждой группе

In [76]:
#вывод таблицы debt_total_income из столбцов 'debt', 'total_income'
debt_total_income = credit[['debt','total_income']]
display(debt_total_income)

,debt,total_income
0,0,253875
1,0,112080
2,0,145885
3,0,267628
4,0,158616
...,...,...
21449,0,224791
21450,0,155999
21451,1,89672
21452,1,244093


In [77]:
#найдем медиану значений столбца 'total_income' и сохраним ее в переменную median_total_income
median_total_income = credit['total_income'].median()
display(median_total_income)

142594.0

In [78]:

def debt_total_income_res(row):
    """"
    создание функции, которая возвращает деление заемщиков на категории, по правилам:
    -'надежный заемщик с доходом выше среднего': debt = 0, total_income > median_total_income
    -'надежный заемщик с доходом средним и ниже среднего': debt = 0, total_income <= median_total_income
    -'НЕнадежный заемщик с доходом выше среднего': debt = 1, total_income > median_total_income
    -'НЕнадежный заемщик с доходом средним и ниже среднего'- во всех остальных случаях
    """
  
    debt = row['debt']
    total_income = row['total_income']
    
    if debt == 0:
        if total_income > median_total_income:
            return 'надежный заемщик с доходом выше среднего'
    if debt == 0:
        if total_income <= median_total_income:
            return 'надежный заемщик с доходом средним и ниже среднего'
    if debt == 1:
        if total_income > median_total_income:
            return 'НЕнадежный заемщик с доходом выше среднего'
    
    return 'НЕнадежный заемщик с доходом средним и ниже среднего'

#проверка работы функции debt_total_income_res:
row_values = [1, 140000]
row_columns = ['debt', 'total_income']
row = pd.Series(data=row_values, index=row_columns)
debt_total_income_res(row)

#создание нового столбца с резульатами функции debt_total_income_res: 
credit['result_dokh'] = credit.apply(debt_total_income_res, axis=1)

# вывод полученных категорий заемщиков и подсчет их количества
display(credit['result_dokh'].value_counts())

надежный заемщик с доходом средним и ниже среднего      9933
надежный заемщик с доходом выше среднего                9780
НЕнадежный заемщик с доходом средним и ниже среднего     910
НЕнадежный заемщик с доходом выше среднего               831
Name: result_dokh, dtype: int64

In [79]:
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem,purpose_res,debt_children_res,result,result_dokh
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.12,"[покупка, , жилье, \n]",недвижимость,надежный заёмщик с детьми,надежный НЕодинокий заёмщик,надежный заемщик с доходом выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.03,"[приобретение, , автомобиль, \n]",автомобиль,надежный заёмщик с детьми,надежный НЕодинокий заёмщик,надежный заемщик с доходом средним и ниже сред...
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.41,"[покупка, , жилье, \n]",недвижимость,надежный бездетный заёмщик,надежный НЕодинокий заёмщик,надежный заемщик с доходом выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.30,"[дополнительный, , образование, \n]",образование,надежный заёмщик с детьми,надежный НЕодинокий заёмщик,надежный заемщик с доходом выше среднего
4,0,2170,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.24,"[сыграть, , свадьба, \n]",свадьба,надежный бездетный заёмщик,надежный НЕодинокий заёмщик,надежный заемщик с доходом выше среднего


In [80]:
#сохраним в переменной:
#z_0_low - надежный заемщик с доходом средним и ниже среднего      
#z_0_hi - надежный заемщик с доходом выше среднего               
#z_1_low - НЕнадежный заемщик с доходом средним и ниже среднего     
#z_1_hi - НЕнадежный заемщик с доходом выше среднего

z_0_hi = credit.loc[credit['result_dokh'] == 'надежный заемщик с доходом выше среднего', 'result_dokh'].count()
display(z_0_hi)
z_0_low = credit.loc[credit['result_dokh'] == 'надежный заемщик с доходом средним и ниже среднего', 'result_dokh'].count()
display(z_0_low)
z_1_hi = credit.loc[credit['result_dokh'] == 'НЕнадежный заемщик с доходом выше среднего', 'result_dokh'].count()
display(z_1_hi)
z_1_low = credit.loc[credit['result_dokh'] == 'НЕнадежный заемщик с доходом средним и ниже среднего', 'result_dokh'].count()
display(z_1_low)



9780

9933

831

910

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> 

Попробуй добавить больше категорий - хотябы 3-4, чтобы вывод был более приближен к реальной жизни.
</div>

In [81]:
#доля ненадежных среди заемщиков с доходом выше среднего
display(z_1_hi/(z_0_hi+z_1_hi))

0.0783149561775516

In [82]:
#доля ненадежных среди заемщиков с доходом средним и ниже среднего
display(z_1_low/(z_0_low+z_1_low))

0.08392511297611362

**Вывод 2.6.3.**

Таким образом:
- доля ненадежных заемщиков с доходом выше среднего - 0.078 (8%)
- доля ненадежных заемщиков с доходом средним и ниже среднего - 0.083 (8%)

То есть доли ненадежных заемщиков среди разных категорий практически одинаковые (среди заемщиков с доходом средним и ниже немного выше), поэтому делаем вывод: от уровня дохода возврат кредита в срок мало зависит. 

2.6.4. ЦЕЛИ КРЕДИТА И ВОЗВРАТ КРЕДИТА В СРОК.

Как разные цели кредита влияют на его возврат в срок?

1) :


In [83]:
#добавим столбец debt (возврат/невозврат кредита в срок)
#в таблицу credit_grouped (с лемматизированным столбцом целей кредита) 
#credit['debt'] = credit['debt']
#credit.head(50)

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> 

1. Как ты определила, что именно такие категории будут самыми популярными? Сделай, пожалуйста, преобразования, которые подсчитают частоту появления слов после лемматизации, чтобы категории выделились наглядно.
2. Добавь эту категоризацию в соответствующий раздел.
</div>

In [84]:
#вывод строк таблицы, где столбец 'debt' имеет значение равное 1 (то есть случаи невозврата кредита) 
#и сохранить ее в переменной debt_purpose_res
debt_purpose_res = credit[credit['debt'] == 1]

#вывод таблицы debt_purpose_res на экран
display(debt_purpose_res)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_years,credit_lem,purpose_res,debt_children_res,result,result_dokh
14,0,1844,56,высшее,0,гражданский брак,1,F,компаньон,1,165127,покупка жилой недвижимости,5.05,"[покупка, , жилой, , недвижимость, \n]",недвижимость,НЕнадежный бездетный заёмщик,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом выше среднего
32,0,4649,34,среднее,1,гражданский брак,1,F,сотрудник,1,139057,на проведение свадьбы,12.74,"[на, , проведение, , свадьба, \n]",свадьба,НЕнадежный бездетный заёмщик,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...
38,0,597,25,высшее,0,не женат / не замужем,4,M,сотрудник,1,192247,образование,1.64,"[образование, \n]",образование,НЕнадежный бездетный заёмщик,НЕнадежный одинокий заёмщик,НЕнадежный заемщик с доходом выше среднего
55,0,4026,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514,сыграть свадьбу,11.03,"[сыграть, , свадьба, \n]",свадьба,НЕнадежный бездетный заёмщик,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...
75,1,2953,38,среднее,1,женат / замужем,0,M,сотрудник,1,81935,операции с недвижимостью,8.09,"[операция, , с, , недвижимость, \n]",недвижимость,НЕнадежный заёмщик с детьми,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21423,0,960,39,среднее,1,гражданский брак,1,M,компаньон,1,176369,дополнительное образование,2.63,"[дополнительный, , образование, \n]",образование,НЕнадежный бездетный заёмщик,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом выше среднего
21440,0,612,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068,покупка жилья для сдачи,1.68,"[покупка, , жилье, , для, , сдача, \n]",недвижимость,НЕнадежный бездетный заёмщик,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...
21444,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,1.28,"[заниматься, , образование, \n]",образование,НЕнадежный заёмщик с детьми,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5.79,"[недвижимость, \n]",недвижимость,НЕнадежный заёмщик с детьми,НЕнадежный НЕодинокий заёмщик,НЕнадежный заемщик с доходом средним и ниже ср...


In [85]:
#группируем таблицу debt_purpose_res по столбцу 'debt_purpose_res'и подсчитываем количество каждой группы
debt_purpose_res = debt_purpose_res.groupby(['purpose_res']).agg({'purpose_res':'count'})

#вывод таблицы debt_purpose_res на экран
display(debt_purpose_res)

,purpose_res
purpose_res,
автомобиль,403
недвижимость,782
образование,370
свадьба,186


<div class="alert alert-success">
<b>Комментарий ревьюера V2:</b> 
    
Суть подсчета был в том, чтобы посмотреть среди лемм какие встреаются значения и каких из них самые популярные. Далее, увидеть, какие в топе и из них выделить категории. Подсчет по самим категориям каректен. Оставим.
</div>

In [86]:
#сортировка таблицы debt_purpose_res по столбцу 'debt_purpose_res' и вывод результата на экран
debt_purpose_res['purpose_res'].sort_values(ascending = False)

purpose_res
недвижимость    782
автомобиль      403
образование     370
свадьба         186
Name: purpose_res, dtype: int64

**Вывод 2.6.4.**

Таким образом, 
- самые частые случаи просрочки выплаты кредита  - это кредиты на: недвижимость и автомобиль;
- меньше случаев просрочки кредита - это кредиты на: организация свадьбы и образование.

Возможно такая ситуация складывается из-за разницы в суммах и сроках, так как кредиты на недвижимость и покупку атомобиля значительно выше, и по суммам и по сроку кредитов на организацию свадьбы и образование.

## Шаг 4. Общий вывод



I. 

На первый взгляд изучая таблицу credit возникли следующие вопросы:

1) Вызвали сомнения данные столбца 'days_employed' (общий трудовой стаж в днях):
- встречаются отрицательные значения
- очень большие числа.

2) Столбец 'education' и 'family_status' заполнен разными регистрами.

3) Столбец 'purpose' - много одинаковых целей кредита записано разными словами.

4) Остальные столбцы сомнений, на первый взгляд, не вызвали.


Исходя из общей информации:

1) В таблице 12 столбцов.
Согласно документации к данным: children — количество детей в семье, days_employed — общий трудовой стаж в днях, dob_years — возраст клиента в годах, education — уровень образования клиента, education_id — идентификатор уровня образования, family_status — семейное положение, family_status_id — идентификатор семейного положения, gender — пол клиента, income_type — тип занятости, debt — имел ли задолженность по возврату кредитов, total_income — ежемесячный доход, purpose — цель получения кредита.
2) Количество значений в столбцах различается - значит, в данных есть пропущенные значения.
3) Столбец 'days_employed' имеет тип float64 - вещественные числа, что является ошибкой, так как трудовой стаж в днях - тип данных должен быть целые числа. 

Согласно информации некоторых числовых показателей таблицы:
1) В столбце 'children' - количество детей, присутствуют аномальные значения: -1 и 20, которые требуют проверки и замены.
 
II.

В процессе предобработки данных были обнаружены пропуски в столбцах: 'days_employed' и 'total_income'. Причем их количество по столбцам было одинаковое. 

Долевое соотношение пропусков получилось равное 10%, поэтому их необходимо было проработать.

- В столбце 'days_employed' пропуски заменили на медианное значение этого столбца в зависимости от возраста (столбец 'dob_years'), чтобы избежать возможных выбросов в столбце, чего не сможет избежать просто среднее значение по столбцу и может дать более искаженные от действительности данные, в отличии от медианы.
-В столбце 'total_income' так же пропуски заменить на медианное значение этого столбца в зависимости от типа занятости 'income_type'.

В столбце 'days_employed' заменили отрицательные значения на положительные, чтобы использовать  их при дальнейших расчетах.

В анализе столбца 'days_employed' замечены аномальный срок стажа - более 900 лет.  Заменили все подобные аномальные значения на медианное значение по всему столбцу 'days_employed'. В связи с такими неадекватно большими колебаниями значений, замена их на медиану была разумнее всего.

Так же в столбце  возраст (’dob_years’)  были обнаружены нулевые значения - что противоречит здравому смыслу. Заменили эти нулевые значения на значение медианы по данному столбцу, так как возраст варьируется от студента до пенсионера, поэтому медиана будет ближе к действительности.

В столбце 'children' обнаружены так же неадекватные значения: -1 и 20.  
-Отрицательные значения, равные: -1, заменили на положительные: 1, чтобы минус не мешал в дальнейших вычислениях.
-Так как в столбце нет больших колебаний значений - заменили значения, равные 20, на среднее значение по столбцу.

III.

В столбцах :

- "общий трудовой стаж в днях" ('days_employed')
- "возраст клиента в годах" ('dob_years') 
- "ежемесячный доход" ('total_income') 

был заменен тип данных на int (целые числа), что соответствует здравому смыслу и упрощения дальнейших рассчетов.

IV.

Приведя все данные в таблице к нижнему регистру, чтобы точнее отфильтровать дубликаты, были удалены дубликаты. 

V.

Провели лемматизацию столбца 'purpose' (цели кредита). Создали отдельный столбец 'credit_lem' c лемматизированными значениями столбца 'purpose'. Эти данные в дальнейшем были использованы для категоризации данных по этому столбцу.

VI.

Провели анализ завсисмости столбцов education и education_id, а так же family_status и family_status_id. Выявили следующие соответствия:

1) 'education' (уровень образования клиента) и 'education_id' (идентификатор уровня образования клиента):
- высшее соответствует цифра 0
- среднее - соотвтетствует цифра 1
- неоконченное высшее - соответствует цифра 2
- начальное - соотвтетствует цифра 3
- ученая степень - соответствует цифра 4

2) 'family_status' (семейное положение) и 'family_status_id' (идентификатор семейного положения):
- в разводе - соответствует 3
- вдовец / вдова - соответствет 2
- гражданский брак - 1
- женат / замужем - 0
- не женат / не замужем - 4

Сгруппировали по категориям столбец purpose (цели кредита):
- недвижимость    10811
- автомобиль       4306
- образование      4013
- свадьба        
и выяснили, что самой популярной покупкой в кредит у населения является покупка недвижимости.

VII.

Провели анализ категорий заемщиков и выяснили самые рискованные группы населения для выдачи кредитов. 

Исходя из анализа данных наличия детей и возврата/невозврата кредита, пришли к выводам: 
Доля ненадежных заемщиков (заещиков, у которых был факт невозврата кредита в срок):
- у бездетных составила - 0.075 (7,5%)
- у заемщиков с детьми - 0.092 (9,2%)
Таким образом, среди заемщиков с детьми процент невозврата кредита в срок больше, чем у бездетных заемщиков.

Анализируя данные семейного положения и факта возврата/невозварат кредита, получили следующие результаты:
- доля ненадежных среди неодиноких заемщиков - 0.08 (8%)
- доля ненадежных среди одиноких заемщиков - 0.09 (9%)

То есть ненадежных среди одиноких заемщиков, хотя и не на много, но все таки больше.

Исследуя доход и надежность заемщиков, пришли к факту:
- доля ненадежных заемщиков с доходом выше среднего - 0.078 (8%)
- доля ненадежных заемщиков с доходом средним и ниже среднего - 0.083 (8%)

То есть доли ненадежных заемщиков среди разных категорий практически одинаковые (среди заемщиков с доходом средним и ниже немного выше), поэтому делаем вывод: от уровня дохода возврат кредита в срок мало зависит. 

И, наконец, изучив как цели получения кредита и факт возврата/невозврата кредита, получили: 
- самые частые случаи просрочки выплаты кредита  - это кредиты на: недвижимость и автомобиль;
- меньше случаев просрочки кредита - это кредиты на: организация свадьбы и образование.



Исходя из всего анализа данных, приведенного выше, можно сделать вывод:  для банка самая рискованная  группа заемщиков - это заемщики,имеющие детей и желающие взять кредит на покупку недвижимости. Возможно такая ситуация складывается из-за разницы в суммах и сроках, так как кредиты на недвижимость и покупку атомобиля значительно выше, и по суммам и по сроку кредитов на организацию свадьбы и образование. То есть кредиты на организацию садьбы и образование более предпочтительнее в плане их надежности, но в другой стороны - они и мало доходные. Проблему более высоких рисков выдачи кредитов на покупку недвижимости и автомобиля банки решают системой страхования.   

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> 

Хороший вывод, расширь его следующими моментами:
    
1. Какие зависимости ты обнаружила в данных при предобработке и что ты сделала, чтобы их привести к нормальному виду?
2. Сделай вывод не только по самой рискованной группе, но и по тем кому предпочтительнее всего давать кредит.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2:</b> 
    
Супер, комментарии получились исчерпывающими. Общий вывод отличный.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.